# Jupyter notebooks

## 1. Imports 

### 1.1. Standard imports

In [8]:
import numpy as np

### 1.2. Non-standard imports

The package `meta-py` (available on PyPI `pip install meta-py`) was used in our sample methods section to provide input data

In [11]:
from metapy import __version__
from metapy.tsp import tsp_io as io
from metapy.tsp.euclidean import gen_matrix, plot_tour 

print(__version__)

0.4.0


## An overview of sections in a experimental report

There are no hard and fast rules for a Jupyter notebook structure that contains computational models and experiments. However, from general experimental science we know that the following standard format works well (adapted from Field and Hole, 2003).

* Title
* Abstract and/or plain English summary
* Introduction
* Method
    * Analysis environment
    * Design
    * Models/Algorithms
    * Case studies
    * Procedure
* Results
* Discussion
* References
* Appendix (may not be needed; potentially links to a separate notebook)

Field and Hole (2003) state that the above structure is relevant for experimental research as it answers the following four questions:

1. **Why** did the authors run the experiments with their models and what did they aim to find? (**introduction**)
2. **How** was the experimental work carried out?  What models were used, how were they implemented (e.g. a 3rd party library or implemented directly); what experimental design was used? (**methods**) 
3. **What** were the **results** of the experiments?
4. **So What?** What do the results means, what are the strengths and limitations of the study. What further work could be completed? 

## 2. An example methods section

In this simple example, a general optimisation algorithm called GRASP is implemented.  It is applied to a famous combinatorial optimisation problem - the traveling salesman problem (TSP).

### 2.1 Methods

In this study we will use a basic form of the GRASP metaheuristic algorithm to solve the symmetric Travelling Salesman Problem (TSP). In the TSP a salesman must visit a set of cities once and only once in the shortest possible distance (or time).  The Greedy Randomised Adaptive Search Procedure (**GRASP**) was introduced by Rio and Resende in 1989. It is an extremely simple multi-start **metaheuristic**. The basic form of GRASP is based on two phases:

1. an initial semi-greedy construction/repair phase; 
2. a period of local search to improve on the initial solution.

The basic GRASP algorithm is outlined below.  Note that the TSP implementation of GRASP in this study always results in feasible solutions.  Optionally GRASP may include a **repair** function used to modify infeasible solutions created during the construction phase.

```
PROCEDURE basic_grasp():
    
    best = []
    do 
    
        s = greedy_construction()
        s = local_search(s)

        if cost(s) > cost(best):
            best = s
    
    until max iterations or time limit reached.
    
    return best
```

#### 2.1.1 Components of GRASP

##### Semi-greedy construction

A fundamental part of GRASP is semi-greedy construction.  In a purely **greedy construction** method for the TSP we select a starting point in the tour and then choose the next closest city (the nearest neighbour algorithm). We continue to do this until all cities are visited. In contrast in **Semi-Greedy construction** we modify the approach to incorporate the probability. Instead of always visiting the closest city there is now a probability that we visit one of the n closest cities.  Therefore semi-greedy algorithms are a half way house between random and greedy search.

##### Restricted Candidate Lists

The first step in semi-greedy construction is to create the restricted candidate list: the list of top valued components that the algorithm will choose from in its next step. In the TSP a straightforward implementation is choosing the closest r cities where r is a hyper-parameter that can be fixed, a random variable or learned over time.

Below we implement the fixed sized 

In [12]:
class FixedRCLSizer:
    '''
    Fixed sized RCL list.
    
    When r = 1 then greedy
    When r = len(tour) then random
    '''
    def __init__(self, r):
        self.r = r
        
    def get_size(self):
        '''
        Returns an int representing the size of the required RCL
        '''
        return self.r

We now introduce a basic semi-greedy implementation in Python.

A key point to remember is that construction is iterative. A new RCL is created in each iteration and a city is chosen from it at random until all cities have been added to the tour.

Here we create a `SemiGreedyConstructor` class. The class accepts a rcl_sizer as an argument (an object that implements a `get_size()` method). This means you can vary the RCL sizing logic and treat it as a hyper parameter.

In [13]:
class SemiGreedyConstructor:
    '''
    Semi-greedy construction of a tour.
    
    For a city i creates a restricted candidate list of size r
    i.e the r shortest distances from city i.  
    Next city is chosen with equal probability.
    Repeats until tour is constructed.
    '''
    def __init__(self, rcl_sizer, tour, matrix,
                 random_seed=None):
        '''
        Constructor
        
        Params:
        ------
        rcl_sizer: object
            sizes the restricted candidate list
        
        tour: np.ndarray
            vector of city indexes included in problem
            
        matrix: np.ndarray
            matrix of travel costs
            
        random_seed: int
            used to control sampling and provides a
            reproducible result.
        '''
        
        # size of rcl
        self.rcl_sizer = rcl_sizer
        
        # cities in a tour
        self.tour = tour
        
        # travel cost matrix
        self.matrix = matrix
        
        # create random number generator
        self.rng = np.random.default_rng(random_seed)
    
    def build(self):
        '''
        Semi-greedy contruction of tour
        
        Returns:
        --------
        np.array
        '''
        # first city in tour
        solution = np.array([self.tour[0]])    
        
        # it is an iterative (construction) procedure
        for i in range(len(self.tour)-1):
            # get the RCL size
            r = self.rcl_sizer.get_size()
            
            # get the RCL
            rcl = self.get_rcl(r, solution, solution[-1])
            
            # select the next city 
            next_city = self.random_from_rcl(rcl)
            
            # update the solution
            solution = np.append(solution, np.array([next_city]))
            
        return solution
    
    def get_rcl(self, r, solution, from_city):
        '''
        Restricted candidate list for final city in current solution
        
        Params:
        -------
        solution: np.ndarray
            vector of current partially constructed solution
            
        from_city: int
            index of city used to construct rcl.
        
        Returns:
        -------
        np.array
        '''
        # get indexes of cities not in solution
        mask = self.tour[~np.in1d(self.tour, solution)]
        
        # get indexes of r smallest travels costs 
        if mask.shape[0] > r:
            # partition the vector for remaining cities - faster than sorting 
            idx = np.argpartition(self.matrix[from_city][mask], 
                                  len(mask) - r)[-r:]
            rcl = mask[idx]
        else:
            # handle when r < n cities remaining 
            rcl = mask
        return rcl
    
    def random_from_rcl(self, rcl):
        '''
        Select a city at random from rcl.
        Return city index in self.matrix
        
        Params:
        -------
        rcl: np.ndarray
            restricted candidate list
            vector of candidate city indexes.
        '''
        return self.rng.choice(rcl)